In [3]:
from ollama_help import check_ollama, check_models
from utils import load_json

if not check_ollama():
    print("Ollama não está instalado ou não está conectado.")
    exit()

questoes = load_json("./questoes/questoes.json")


In [2]:
# Questões de Texto

text_models = ["phi4", "llava", "deepseek-r1", "llama3.2", "gemma2", "mistral"]
avaliable_text_models = list(check_models(text_models))

text_questions   = list(filter(lambda x : x["type"] == "only-text", questoes))

print(f"Total de {len(text_questions)} questões apenas texto")

✅ phi4 disponível
✅ llava disponível
✅ deepseek-r1 disponível
✅ llama3.2 disponível
✅ gemma2 disponível
✅ mistral disponível
Total de 1608 questões apenas texto


In [3]:
# Teste das Questões no Ollama

from ollama_help.runner import test_ollama_models
from utils import test_table

test_ollama_models(
    models = sorted(avaliable_text_models, key=lambda x : len(x)),
    questions = text_questions,
    predict_file ="./predict_data/text_predictions.json",
)

test_table("./predict_data/text_predictions.json", len(text_questions))

,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,deepseek-r1,1609 (100%),883,127,599,55%,09:06:24,ND,20.38,07:49,2.53
1,llama3.2,1609 (100%),426,28,1155,26%,08:53,ND,0.33,41.73,0.09
2,llava,1609 (100%),730,77,802,45%,15:22,ND,0.57,12.12,0.15
3,mistral,1609 (100%),931,30,648,58%,24:38,ND,0.92,20.85,0.17
4,phi4,1609 (100%),1337,12,260,83%,02:54:06,ND,6.49,01:46,0.32
5,gemma2,1609 (100%),1208,30,371,75%,16:39,ND,0.62,11.38,0.27
6,TOTAL,9654 (100%),5515,304,3835,57%,13:06:05,ND,4.89,07:49,0.09


In [4]:
visions_models = ["llava", "llama3.2-vision", "llava-llama3", "minicpm-v","moondream", "llava-phi3"]
available_visions_models = list(check_models(visions_models))

context_questions = list(filter(lambda x : x["type"] == "context-image", questoes))

print(f"Total de {len(context_questions)} questões com imagem de contexto")

✅ llava disponível
✅ llama3.2-vision disponível
✅ llava-llama3 disponível
✅ minicpm-v disponível
✅ moondream disponível
✅ llava-phi3 disponível
Total de 795 questões com imagem de contexto


In [ ]:
test_ollama_models(
    models = sorted(available_visions_models, key=lambda x : len(x)),
    questions = context_questions,
    predict_file ="./predict_data/correct_context.json",
)

# Error ao gerar resposta para a pergunta 2016076 do modelo minicpm-v: POST predict: Post "http://127.0.0.1:50544/completion": read tcp 127.0.0.1:50546->127.0.0.1:50544: wsarecv: An existing connection was forcibly closed by the remote host. (status code: 500)

test_table("./predict_data/correct_context.json", len(context_questions))


,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,llava,795 (100%),154,179,462,19%,20:49,0.00,1.57,19.91,0.28
1,minicpm-v,794 (100%),233,64,497,29%,24:16,1.83,1.83,14.82,0.30
2,moondream,795 (100%),8,770,17,1%,04:57,0.00,0.37,4.51,0.10
3,llava-phi3,795 (100%),182,132,481,23%,14:51,0.00,1.12,20.50,0.18
4,llava-llama3,795 (100%),91,413,291,11%,10:08,0.00,0.77,6.72,0.22
5,llama3.2-vision,206 (26%),74,1,131,36%,40:14,01:55:04,11.72,01:18,0.64
6,TOTAL,4180 (88%),742,1559,1879,18%,01:55:18,ND,1.66,01:18,0.10


In [ ]:
from ollama_help.runner import test_ollama_multi_models
import random

best_text_models = ["mistral", "gemma2", "phi4"]
random.seed(42)

# Coletando 25 Questões aleatórias de cada uma das disciplinas
example_context_questions = (
    list(random.sample(list(filter(lambda x : x["discipline"] == "ciencias-humanas", context_questions)), 25))[:10] +
    list(random.sample(list(filter(lambda x : x["discipline"] == "ciencias-natureza", context_questions)),25))[:10] +
    list(random.sample(list(filter(lambda x : x["discipline"] == "linguagens", context_questions)),25))[:10] +
    list(random.sample(list(filter(lambda x : x["discipline"] == "matematica", context_questions)),25))[:10]
)

test_ollama_multi_models(
    text_models = sorted(best_text_models),
    vision_models = sorted(visions_models),
    questions= example_context_questions,
    predict_file ="./predict_data/multi_models_context_prediction.json",
)

test_table("./predict_data/multi_models_context_prediction.json", len(example_context_questions))

,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,llama3.2-vision+gemma2,75 (188%),36,3,36,48%,39:29,-1105.92,31.60,56.25,12.09
1,llama3.2-vision+mistral,40 (100%),13,1,26,32%,21:13,0.00,31.85,01:05,10.46
2,llama3.2-vision+phi4,40 (100%),24,0,16,60%,23:29,0.00,35.25,55.78,11.77
3,llava+gemma2,40 (100%),20,1,19,50%,05:39,0.00,8.48,14.25,5.87
4,llava+mistral,40 (100%),9,6,25,22%,06:31,0.00,9.79,23.85,5.13
5,llava+phi4,40 (100%),21,0,19,52%,11:53,0.00,17.84,38.28,8.34
6,llava-llama3+gemma2,40 (100%),18,0,22,45%,06:59,0.00,10.48,17.60,6.90
7,llava-llama3+mistral,40 (100%),15,5,20,38%,07:06,0.00,10.66,18.55,6.67
8,llava-llama3+phi4,40 (100%),21,0,19,52%,13:28,0.00,20.22,58.21,9.69
9,llava-phi3+gemma2,40 (100%),19,0,21,48%,05:38,0.00,8.46,13.89,5.49


In [ ]:
# Questões com alternativas com Imagem (Apenas Visão)

answer_questions = list(filter(lambda x :x['type'] == 'answer-image', questoes))

test_ollama_models(
    models = ["moondream", "minicpm-v", "llava-phi3"], # o llama3.2-vision não suporta mais de uma imagem
    questions = answer_questions,
    predict_file = "./predict_data/answ-vision.json"
)

test_table("./predict_data/answ-vision.json", len(answer_questions))

,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,moondream,61 (100%),0,61,0,0%,01:08,0.00,1.12,4.57,1.02
1,minicpm-v,61 (100%),6,13,42,10%,01:58,0.00,1.94,7.37,1.41
2,llava-phi3,61 (100%),0,61,0,0%,03:06,0.00,3.06,11.77,2.12
3,TOTAL,183 (100%),6,135,42,3%,06:13,ND,2.04,11.77,1.02


In [ ]:
# Questões com alternativas com Imagem (Multi Modelos)
from ollama_help.runner import test_ollama_multi_models

answer_questions = list(filter(lambda x :x['type'] == 'answer-image', questoes))

test_ollama_multi_models(
    text_models = ['phi4', 'gemma2'],
    vision_models= ["llama3.2-vision", "moondream", "minicpm-v"],
    questions=answer_questions,
    predict_file="./predict_data/answ-multi.json"
)

test_table("./predict_data/answ-multi.json",len(answer_questions))

,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,llama3.2-vision+phi4,61 (100%),9,8,44,15%,11:17,0.00,11.10,26.76,2.26
1,llama3.2-vision+gemma2,61 (100%),11,3,47,18%,01:41,0.00,1.67,9.11,0.27
2,moondream+phi4,61 (100%),10,10,41,16%,10:52,0.00,10.69,25.77,2.56
3,moondream+gemma2,61 (100%),11,5,45,18%,01:57,0.00,1.93,8.81,0.27
4,minicpm-v+phi4,61 (100%),11,13,37,18%,10:31,0.00,10.35,25.40,2.12
5,minicpm-v+gemma2,61 (100%),14,4,43,23%,01:45,0.00,1.72,8.52,0.27
6,TOTAL,366 (100%),66,43,257,18%,38:05,ND,6.24,26.76,0.27


,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,llama3.2-vision+phi4,61 (100%),9,8,44,15%,11:17,0.00,11.10,26.76,2.26
1,llama3.2-vision+gemma2,61 (100%),11,3,47,18%,01:41,0.00,1.67,9.11,0.27
2,moondream+phi4,61 (100%),10,10,41,16%,10:52,0.00,10.69,25.77,2.56
3,moondream+gemma2,61 (100%),11,5,45,18%,01:57,0.00,1.93,8.81,0.27
4,minicpm-v+phi4,61 (100%),11,13,37,18%,10:31,0.00,10.35,25.40,2.12
5,minicpm-v+gemma2,61 (100%),14,4,43,23%,01:45,0.00,1.72,8.52,0.27
6,TOTAL,366 (100%),66,43,257,18%,38:05,ND,6.24,26.76,0.27


In [ ]:
# Questões com imagens no contexto e nas alternativas (Vision Models)

full_image_questions = list(filter(lambda x :x['type'] == 'full-image', questoes))

test_ollama_models(
    models = ["moondream", "minicpm-v", "llava-phi3"],
    questions = full_image_questions,
    predict_file = "./predict_data/full-vision.json"
)

test_table("./predict_data/full-vision.json", len(full_image_questions))

,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,moondream,65 (100%),0,65,0,0%,01:16,0.00,1.18,1.38,1.07
1,minicpm-v,65 (100%),4,17,44,6%,03:41,0.00,3.41,8.49,1.51
2,llava-phi3,65 (100%),0,65,0,0%,04:22,0.00,4.04,9.37,2.66
3,TOTAL,195 (100%),4,147,44,2%,09:21,ND,2.88,9.37,1.07


,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,moondream,65 (100%),0,65,0,0%,01:16,0.00,1.18,1.38,1.07
1,minicpm-v,65 (100%),4,17,44,6%,03:41,0.00,3.41,8.49,1.51
2,llava-phi3,65 (100%),0,65,0,0%,04:22,0.00,4.04,9.37,2.66
3,TOTAL,195 (100%),4,147,44,2%,09:21,ND,2.88,9.37,1.07


In [4]:
# Questoes com imagem no contexto e nas alternativas (Multi Modelos)
from ollama_help.runner import test_ollama_multi_models

full_image_questions = list(filter(lambda x :x['type'] == 'full-image', questoes))


test_ollama_multi_models(
    text_models = ['phi4', 'gemma2'],
    vision_models= ["llama3.2-vision", "moondream", "minicpm-v"],
    questions=full_image_questions,
    predict_file="./predict_data/full-multi.json"
)

test_table("./predict_data/full-multi.json",len(full_image_questions))

,Model,Finsh,OK,Null,Err,Acc,Ttot,Tle,Tavg,Tmax,Tmin
0,llama3.2-vision+phi4,12 (18%),1,0,11,8%,31:36,02:19:35,02:38,05:13,01:27
1,llama3.2-vision+gemma2,0 (0%),0,0,0,0%,0.00,0.00,0.00,0.00,ND
2,moondream+phi4,0 (0%),0,0,0,0%,0.00,0.00,0.00,0.00,ND
3,moondream+gemma2,0 (0%),0,0,0,0%,0.00,0.00,0.00,0.00,ND
4,minicpm-v+phi4,0 (0%),0,0,0,0%,0.00,0.00,0.00,0.00,ND
5,minicpm-v+gemma2,0 (0%),0,0,0,0%,0.00,0.00,0.00,0.00,ND
6,TOTAL,12 (3%),1,0,11,8%,31:36,02:19:35,02:38,05:13,01:27


KeyboardInterrupt: 